In [7]:
import pandas as pd
import numpy as np
import gensim
from sklearn.datasets import fetch_20newsgroups
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from sklearn import preprocessing
import nltk
import re
from sklearn.model_selection import GridSearchCV
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,accuracy_score
import itertools 

In [8]:
def myTokenizer(text):
    return nltk.regexp_tokenize(text, "\\b[a-zA-Z]{3,}\\b")
def tokenizeContents(contents):
    regex = r'\b\w+\b'
    li=re.findall(regex,contents)
    return [myTokenizer(content) for content in li]
def preprocess():
    model = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\Arfat\Downloads\Compressed\GoogleNews-vectors-negative300.bin.gz', binary=True)
    twenty_all = fetch_20newsgroups(subset='all', shuffle=True,remove=('headers', 'footers', 'quotes'))
    full1=[" ".join(data.split("\n")) for data in twenty_all.data]
    stopWords = set(stopwords.words('english'))
    full= []
    tmp=[] 
    for d in range(len(full1)):
        words = word_tokenize(full1[d])
        for w in words:
            if w not in stopWords:
                tmp.append(w)
        x=" ".join(tmp)
        tmp=myTokenizer(x)
        x=" ".join(tmp)
        tmp=[]
        full.append(x)
    y=twenty_all.target
    X,y=readydocs(full,y,model)
    return X,y
def readydocs(full,y,model):
    add=np.zeros(300,dtype='float32')
    doc=np.zeros((18846,300))
    l=[]#for empty documents
    count=0 #total times the key error is raised
    c_names=[] #contains words that are in documents but they don't have vector representations
    count_word_exists=0 #checks number of words having vector representations in the current document
    del_index=[] # has index of the documnets where no word has a vector representation

    rate=[] #hit rate for words having vector representations in a document
    for q1 in range(len(full)):
        token=tokenizeContents(full[q1])
        token = list(filter(None, token))
        if len(token)==0:
            l.append(q1)
            rate.append(0)
            doc[q1]=np.zeros(300,dtype='float32')
            continue
        for q in range(len(token)):
            try:
                add=add+model.word_vec("".join(token[q]))
                count_word_exists=count_word_exists+1

            except KeyError:
                count+=1
                if "".join(token[q]) not in c_names:
                    c_names.append("".join(token[q]))
                continue
    #print(add,len(token))
        if count_word_exists==0:
            del_index.append(q1)
            add=np.zeros(300,dtype='float32')
            doc[q1]=add
            rate.append(0)
            continue
        rate.append(count_word_exists/len(token))
        add=add/count_word_exists
        doc[q1]=add
        add=np.zeros(300,dtype='float32')
        count_word_exists=0
    doc=[ value for (i, value) in enumerate(doc) if i not in set(l+del_index) ]
    doc=np.asarray(doc)
    X,y=delundocs(full,y,l,del_index)###delete unnecessary docs
    return X,y
def delundocs(full,y,l,del_index):
    #y=twenty_all.target                                 ##############deleting docs for all methods
    full=[ value for (i, value) in enumerate(full) if i not in set(l+del_index) ]
    full_f=full
    full=np.asarray(full)
    for i in sorted(set(l+del_index),reverse=True):
        y=np.delete(y,i)
    return full_f,y

In [9]:
def ftext(k,X,y,train_index,test_index,dim=100,epoch=50,ws=5,lr=0.1):
    ##################              Fast text
    cmd1="fasttext.exe supervised " ###############################################start
    cmd1=cmd1+' -dim '+str(dim)+' -epoch '+str(epoch)+' -ws '+str(ws)+' -lr '+str(lr)+" -input train20.txt -output model20 >> ftrainresult.txt"
    #print (cmd1)
    #return
    cmd2="fasttext test model20.bin test20.txt >> ftestresult.txt"
    cmd3="fasttext predict model20.bin test20.txt > result20.txt"
    c=0
    outF = open("train20.txt", "w")
    out1 = open("test20.txt", "w")
    for i in range(len(train_index)):
        outF.write("__label__")
        outF.write(str(y[train_index[i]]))
        outF.write(" ")
        try :
            outF.write(X[train_index[i]])
            outF.write("\n")
        except UnicodeEncodeError:
            print("i",i)
            outF.write("\n")
            continue
    os.system(cmd1)
    print(len(test_index))
    c=0
    for j in range(len(test_index)):
        out1.write("__label__")
        out1.write(str(y[test_index[j]]))
        out1.write(" ")
        c=c+1
        try :
            out1.write(X[test_index[j]])
            out1.write("\n")
        except UnicodeEncodeError:
            print("j",j)
            out1.write("\n")
            continue
    outF.close()
    out1.close()
    os.system(cmd2)
    os.system(cmd3)
    f = open('result20.txt', 'r')
    lines = np.asarray([np.int32("".join(re.findall('\d+', line))) for line in f.readlines()])
    f.close()
    #self.trupresav("fasttext"+str(k),lines,test_index)
    #self.report=classification_report(self.y[test_index],lines,target_names=self.twenty_all.target_names)
    #self.reporttocsv('fasttext',i=k)
    print("Accuracy score :",round(accuracy_score(y[test_index],lines),3))
    return round(accuracy_score(y[test_index],lines),3)
    

In [10]:
dim_val=[10,25,50,100,200,250,300]
epoch_val=[5,20,25,50,100]
ws_val=[5,10]
lr_val=[0.05,0.025,0.075,0.1,0.150,0.175]
p_grid=dict(dim=dim_val,epoch=epoch_val,ws=ws_val,lr=lr_val)


In [11]:
CP=[dict(zip(p_grid, x)) for x in itertools.product(*p_grid.values())]


In [12]:
X,y=preprocess()

In [13]:
skf = StratifiedKFold(n_splits=3,shuffle=True,random_state=1)

In [14]:
ac_list=[]
for x in CP:
    for k,(train_index, test_index) in enumerate(skf.split(X, y)):
        ac_list.append(ftext(k,X,y,train_index,test_index,x['dim'],x['epoch'],x['ws'],x['lr']))
    

6105
Accuracy score : 0.136
6101
Accuracy score : 0.101
6092
Accuracy score : 0.143
6105
Accuracy score : 0.116
6101
Accuracy score : 0.109
6092
Accuracy score : 0.117
6105
Accuracy score : 0.177
6101
Accuracy score : 0.168
6092
Accuracy score : 0.159
6105
Accuracy score : 0.221
6101
Accuracy score : 0.231
6092
Accuracy score : 0.21
6105
Accuracy score : 0.411
6101
Accuracy score : 0.434
6092
Accuracy score : 0.434
6105
Accuracy score : 0.493
6101
Accuracy score : 0.511
6092
Accuracy score : 0.501
6105
Accuracy score : 0.135
6101
Accuracy score : 0.101
6092
Accuracy score : 0.145
6105
Accuracy score : 0.114
6101
Accuracy score : 0.11
6092
Accuracy score : 0.116
6105
Accuracy score : 0.184
6101
Accuracy score : 0.167
6092
Accuracy score : 0.158
6105
Accuracy score : 0.225
6101
Accuracy score : 0.234
6092
Accuracy score : 0.209
6105
Accuracy score : 0.408
6101
Accuracy score : 0.435
6092
Accuracy score : 0.433
6105
Accuracy score : 0.488
6101
Accuracy score : 0.505
6092
Accuracy score : 

6105
Accuracy score : 0.702
6101
Accuracy score : 0.697
6092
Accuracy score : 0.709
6105
Accuracy score : 0.704
6101
Accuracy score : 0.699
6092
Accuracy score : 0.709
6105
Accuracy score : 0.702
6101
Accuracy score : 0.701
6092
Accuracy score : 0.709
6105
Accuracy score : 0.701
6101
Accuracy score : 0.698
6092
Accuracy score : 0.707
6105
Accuracy score : 0.695
6101
Accuracy score : 0.689
6092
Accuracy score : 0.706
6105
Accuracy score : 0.628
6101
Accuracy score : 0.626
6092
Accuracy score : 0.641
6105
Accuracy score : 0.7
6101
Accuracy score : 0.697
6092
Accuracy score : 0.709
6105
Accuracy score : 0.702
6101
Accuracy score : 0.699
6092
Accuracy score : 0.71
6105
Accuracy score : 0.702
6101
Accuracy score : 0.697
6092
Accuracy score : 0.707
6105
Accuracy score : 0.702
6101
Accuracy score : 0.699
6092
Accuracy score : 0.707
6105
Accuracy score : 0.703
6101
Accuracy score : 0.699
6092
Accuracy score : 0.71
6105
Accuracy score : 0.694
6101
Accuracy score : 0.69
6092
Accuracy score : 0.7

6105
Accuracy score : 0.697
6101
Accuracy score : 0.694
6092
Accuracy score : 0.705
6105
Accuracy score : 0.697
6101
Accuracy score : 0.697
6092
Accuracy score : 0.706
6105
Accuracy score : 0.495
6101
Accuracy score : 0.51
6092
Accuracy score : 0.511
6105
Accuracy score : 0.188
6101
Accuracy score : 0.165
6092
Accuracy score : 0.176
6105
Accuracy score : 0.643
6101
Accuracy score : 0.647
6092
Accuracy score : 0.654
6105
Accuracy score : 0.68
6101
Accuracy score : 0.685
6092
Accuracy score : 0.692
6105
Accuracy score : 0.697
6101
Accuracy score : 0.694
6092
Accuracy score : 0.705
6105
Accuracy score : 0.696
6101
Accuracy score : 0.695
6092
Accuracy score : 0.705
6105
Accuracy score : 0.595
6101
Accuracy score : 0.604
6092
Accuracy score : 0.612
6105
Accuracy score : 0.239
6101
Accuracy score : 0.231
6092
Accuracy score : 0.226
6105
Accuracy score : 0.677
6101
Accuracy score : 0.677
6092
Accuracy score : 0.687
6105
Accuracy score : 0.69
6101
Accuracy score : 0.69
6092
Accuracy score : 0.

6101
Accuracy score : 0.697
6092
Accuracy score : 0.708
6105
Accuracy score : 0.691
6101
Accuracy score : 0.688
6092
Accuracy score : 0.7
6105
Accuracy score : 0.701
6101
Accuracy score : 0.698
6092
Accuracy score : 0.706
6105
Accuracy score : 0.702
6101
Accuracy score : 0.698
6092
Accuracy score : 0.707
6105
Accuracy score : 0.701
6101
Accuracy score : 0.696
6092
Accuracy score : 0.707
6105
Accuracy score : 0.7
6101
Accuracy score : 0.698
6092
Accuracy score : 0.705
6105
Accuracy score : 0.162
6101
Accuracy score : 0.167
6092
Accuracy score : 0.151
6105
Accuracy score : 0.241
6101
Accuracy score : 0.274
6092
Accuracy score : 0.263
6105
Accuracy score : 0.161
6101
Accuracy score : 0.092
6092
Accuracy score : 0.125
6105
Accuracy score : 0.174
6101
Accuracy score : 0.159
6092
Accuracy score : 0.157
6105
Accuracy score : 0.273
6101
Accuracy score : 0.265
6092
Accuracy score : 0.262
6105
Accuracy score : 0.363
6101
Accuracy score : 0.356
6092
Accuracy score : 0.358
6105
Accuracy score : 0.

6101
Accuracy score : 0.67
6092
Accuracy score : 0.679
6105
Accuracy score : 0.688
6101
Accuracy score : 0.687
6092
Accuracy score : 0.698
6105
Accuracy score : 0.695
6101
Accuracy score : 0.693
6092
Accuracy score : 0.704
6105
Accuracy score : 0.695
6101
Accuracy score : 0.695
6092
Accuracy score : 0.706
6105
Accuracy score : 0.689
6101
Accuracy score : 0.686
6092
Accuracy score : 0.698
6105
Accuracy score : 0.569
6101
Accuracy score : 0.577
6092
Accuracy score : 0.582
6105
Accuracy score : 0.697
6101
Accuracy score : 0.695
6092
Accuracy score : 0.705
6105
Accuracy score : 0.697
6101
Accuracy score : 0.698
6092
Accuracy score : 0.707
6105
Accuracy score : 0.7
6101
Accuracy score : 0.696
6092
Accuracy score : 0.707
6105
Accuracy score : 0.7
6101
Accuracy score : 0.695
6092
Accuracy score : 0.705
6105
Accuracy score : 0.689
6101
Accuracy score : 0.685
6092
Accuracy score : 0.698
6105
Accuracy score : 0.569
6101
Accuracy score : 0.576
6092
Accuracy score : 0.583
6105
Accuracy score : 0.6

In [35]:
import statistics as st
ac_conlis=[]
for i in range(0,len(ac_list),3):
    ac_conlis.append(st.mean([ac_list[i],ac_list[i+1],ac_list[i+2]]))

In [36]:
index=np.argsort(ac_conlis)[::-1][:50]

In [41]:
CP=np.asarray(CP)
ac_conlis=np.asarray(ac_conlis)
for v in index:
    print('parameters:',CP[v],'-------------------->>>>score :',round(ac_conlis[v],2))

parameters: {'dim': 100, 'epoch': 100, 'ws': 10, 'lr': 0.075} -------------------->>>>score : 0.7
parameters: {'dim': 25, 'epoch': 100, 'ws': 5, 'lr': 0.1} -------------------->>>>score : 0.7
parameters: {'dim': 25, 'epoch': 100, 'ws': 10, 'lr': 0.05} -------------------->>>>score : 0.7
parameters: {'dim': 100, 'epoch': 100, 'ws': 5, 'lr': 0.1} -------------------->>>>score : 0.7
parameters: {'dim': 25, 'epoch': 50, 'ws': 5, 'lr': 0.15} -------------------->>>>score : 0.7
parameters: {'dim': 100, 'epoch': 50, 'ws': 5, 'lr': 0.15} -------------------->>>>score : 0.7
parameters: {'dim': 25, 'epoch': 100, 'ws': 5, 'lr': 0.05} -------------------->>>>score : 0.7
parameters: {'dim': 25, 'epoch': 100, 'ws': 10, 'lr': 0.1} -------------------->>>>score : 0.7
parameters: {'dim': 25, 'epoch': 50, 'ws': 5, 'lr': 0.1} -------------------->>>>score : 0.7
parameters: {'dim': 25, 'epoch': 100, 'ws': 5, 'lr': 0.15} -------------------->>>>score : 0.7
parameters: {'dim': 25, 'epoch': 100, 'ws': 5, 'lr